## Import Libraries

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings #Ignore all warnings
warnings.filterwarnings("ignore")

import itertools

from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler, OneHotEncoder, PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_regression, mutual_info_regression
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

## Import & Join Datasets

In [3]:
clinical_data = pd.read_csv('Coding_Assignment_Files/clinical_data.csv')
demo_data = pd.read_csv('Coding_Assignment_Files/demographics.csv')
billid_data = pd.read_csv('Coding_Assignment_Files/bill_id.csv')
billamt_data = pd.read_csv('Coding_Assignment_Files/bill_amount.csv')

In [4]:
# Check Shape
print(clinical_data.shape)
print(demo_data.shape)
print(billid_data.shape)
print(billamt_data.shape)

(3400, 26)
(3000, 5)
(13600, 3)
(13600, 2)


In [ ]:
# Join bill_id to bill amount by bill_id - new table name 'Bills'
# Join Bills to clinical_data by id/patient_id- new table name 'Patient Bills'
# Join Patient Bills to demographic by patient_id - new table name 'treatment_df'


In [5]:
bills_df = billid_data.merge(billamt_data, on='bill_id', how='left')
bills_df.head()

,bill_id,patient_id,date_of_admission,amount
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01,951.214467
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,19900.297086
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01,12338.269972
3,3762633379,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,1145.372193
4,7654730355,1d21f2be18683991eb93d182d6b2d220,2011-01-01,1133.601902


In [6]:
# Change 'id' to 'patient_id'
clinical_data = clinical_data.rename(columns = {"id":'patient_id'})
clinical_data.head()

,patient_id,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,1d21f2be18683991eb93d182d6b2d220,2011-01-01,2011-01-11,0,1.0,0,0,0.0,0,0,...,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
1,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,2011-01-11,0,0.0,0,0,0.0,0,0,...,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
2,c85cf97bc6307ded0dd4fef8bad2fa09,2011-01-02,2011-01-13,0,0.0,0,0,0.0,0,0,...,1,1,1,1,0,11.2,26.2,100.0,72.0,151.0
3,e0397dd72caf4552c5babebd3d61736c,2011-01-02,2011-01-14,0,1.0,No,0,0.0,1,1,...,1,1,1,1,1,13.3,28.4,76.0,64.4,152.0
4,94ade3cd5f66f4584902554dff170a29,2011-01-08,2011-01-16,0,0.0,No,0,0.0,1,1,...,0,1,0,1,0,12.0,27.8,87.0,55.6,160.0


In [9]:
patients_df = demo_data.merge(clinical_data, on='patient_id', how='left')
patients_df.head()

,patient_id,gender,race,resident_status,date_of_birth,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,fa2d818b2261e44e30628ad1ac9cc72c,Female,Indian,Singaporean,1971-05-14,2014-09-20,2014-10-04,0,0.0,0,...,1,1,0,0,1,14.2,27.0,81.0,68.1,161.0
1,5b6477c5de78d0b138e3b0c18e21d0ae,f,Chinese,Singapore citizen,1976-02-18,2014-08-08,2014-08-18,0,0.0,No,...,1,1,1,1,1,14.3,27.2,92.0,76.5,151.0
2,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singapore citizen,1982-07-03,2011-03-10,2011-03-20,0,0.0,No,...,0,0,1,1,1,14.4,26.6,92.0,69.7,175.0
3,320aa16c61937447fd6631bf635e7fde,Male,Chinese,Singapore citizen,1982-07-03,2013-07-02,2013-07-14,0,0.0,0,...,1,0,0,1,0,18.0,26.6,92.0,66.7,175.0
4,c7f3881684045e6c49020481020fae36,Male,Malay,Singapore citizen,1947-06-15,2015-10-28,2015-11-10,0,1.0,0,...,1,1,1,1,1,14.9,26.7,97.0,96.9,180.0


In [11]:
# Join on patient_id and date of admission
treatment_df = bills_df.merge(patients_df, on=['patient_id', 'date_of_admission',], how='left')
treatment_df.head()

,bill_id,patient_id,date_of_admission,amount,gender,race,resident_status,date_of_birth,date_of_discharge,medical_history_1,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,7968360812,1d21f2be18683991eb93d182d6b2d220,2011-01-01,951.214467,Male,Indian,Singaporean,1976-12-19,2011-01-11,0,...,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
1,6180579974,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,19900.297086,Female,Malay,Singaporean,1978-07-20,2011-01-11,0,...,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
2,7512568183,1d21f2be18683991eb93d182d6b2d220,2011-01-01,12338.269972,Male,Indian,Singaporean,1976-12-19,2011-01-11,0,...,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0
3,3762633379,62bdca0b95d97e99e1c712048fb9fd09,2011-01-01,1145.372193,Female,Malay,Singaporean,1978-07-20,2011-01-11,0,...,0,0,1,1,1,13.8,22.6,89.0,78.4,160.0
4,7654730355,1d21f2be18683991eb93d182d6b2d220,2011-01-01,1133.601902,Male,Indian,Singaporean,1976-12-19,2011-01-11,0,...,0,0,0,1,1,13.2,30.9,123.0,71.3,161.0


In [13]:
treatment_df = treatment_df.groupby(['bill_id', 'patient_id','amount'], as_index=False).apply(lambda x : x.sum())
treatment_df.head()

,,,bill_id,patient_id,date_of_admission,amount,gender,race,resident_status,date_of_birth,date_of_discharge,medical_history_1,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
bill_id,patient_id,amount,,,,,,,,,,,,,,,,,,,,,
558412,54c00a685cd49f4ed59a389657af655b,6539.159300,558412,54c00a685cd49f4ed59a389657af655b,2013-02-23,6539.159300,Female,Chinese,Singaporean,1974-05-09,2013-03-04,0,...,0,1,0,1,0,14.3,26.4,113.0,81.0,161.0
1219378,a135b44a73f9e99424a91edc846185de,876.424366,1219378,a135b44a73f9e99424a91edc846185de,2013-11-28,876.424366,Male,Chinese,Singapore citizen,1946-02-09,2013-12-11,1,...,0,0,0,1,1,16.0,25.9,91.0,89.9,163.0
3568195,4a8da205ade786dd74c316c39dcbe99f,956.968933,3568195,4a8da205ade786dd74c316c39dcbe99f,2014-12-12,956.968933,Male,Chinese,Singaporean,1969-11-13,2014-12-25,0,...,0,1,1,1,1,11.9,29.9,85.0,77.8,176.0
3722123,277ad90d43a1289b0bf22f9795c5c706,836.612493,3722123,277ad90d43a1289b0bf22f9795c5c706,2012-02-16,836.612493,Female,chinese,Singaporean,1973-04-21,2012-02-27,1,...,1,0,0,1,1,11.2,27.6,77.0,80.2,151.0
4477075,a54070aa18ab3579495a157364f70a39,397.293041,4477075,a54070aa18ab3579495a157364f70a39,2012-12-03,397.293041,Male,Chinese,Singaporean,1968-09-05,2012-12-14,1,...,1,1,1,1,0,15.0,27.6,108.0,87.0,175.0


In [31]:
treatment_df = treatment_df.rename_axis(index={'bill_id':'a', 'patient_id':'b', 'amount':'c'})
treatment_df.head()

,,,bill_id,patient_id,date_of_admission,amount,gender,race,resident_status,date_of_birth,date_of_discharge,medical_history_1,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
a,b,c,,,,,,,,,,,,,,,,,,,,,
558412,54c00a685cd49f4ed59a389657af655b,6539.159300,558412,54c00a685cd49f4ed59a389657af655b,2013-02-23,6539.159300,Female,Chinese,Singaporean,1974-05-09,2013-03-04,0,...,0,1,0,1,0,14.3,26.4,113.0,81.0,161.0
1219378,a135b44a73f9e99424a91edc846185de,876.424366,1219378,a135b44a73f9e99424a91edc846185de,2013-11-28,876.424366,Male,Chinese,Singapore citizen,1946-02-09,2013-12-11,1,...,0,0,0,1,1,16.0,25.9,91.0,89.9,163.0
3568195,4a8da205ade786dd74c316c39dcbe99f,956.968933,3568195,4a8da205ade786dd74c316c39dcbe99f,2014-12-12,956.968933,Male,Chinese,Singaporean,1969-11-13,2014-12-25,0,...,0,1,1,1,1,11.9,29.9,85.0,77.8,176.0
3722123,277ad90d43a1289b0bf22f9795c5c706,836.612493,3722123,277ad90d43a1289b0bf22f9795c5c706,2012-02-16,836.612493,Female,chinese,Singaporean,1973-04-21,2012-02-27,1,...,1,0,0,1,1,11.2,27.6,77.0,80.2,151.0
4477075,a54070aa18ab3579495a157364f70a39,397.293041,4477075,a54070aa18ab3579495a157364f70a39,2012-12-03,397.293041,Male,Chinese,Singaporean,1968-09-05,2012-12-14,1,...,1,1,1,1,0,15.0,27.6,108.0,87.0,175.0


In [33]:
treatment_df = treatment_df.droplevel([0,1]).reset_index()
treatment_df.head()

,c,bill_id,patient_id,date_of_admission,amount,gender,race,resident_status,date_of_birth,date_of_discharge,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
0,6539.159300,558412,54c00a685cd49f4ed59a389657af655b,2013-02-23,6539.159300,Female,Chinese,Singaporean,1974-05-09,2013-03-04,...,0,1,0,1,0,14.3,26.4,113.0,81.0,161.0
1,876.424366,1219378,a135b44a73f9e99424a91edc846185de,2013-11-28,876.424366,Male,Chinese,Singapore citizen,1946-02-09,2013-12-11,...,0,0,0,1,1,16.0,25.9,91.0,89.9,163.0
2,956.968933,3568195,4a8da205ade786dd74c316c39dcbe99f,2014-12-12,956.968933,Male,Chinese,Singaporean,1969-11-13,2014-12-25,...,0,1,1,1,1,11.9,29.9,85.0,77.8,176.0
3,836.612493,3722123,277ad90d43a1289b0bf22f9795c5c706,2012-02-16,836.612493,Female,chinese,Singaporean,1973-04-21,2012-02-27,...,1,0,0,1,1,11.2,27.6,77.0,80.2,151.0
4,397.293041,4477075,a54070aa18ab3579495a157364f70a39,2012-12-03,397.293041,Male,Chinese,Singaporean,1968-09-05,2012-12-14,...,1,1,1,1,0,15.0,27.6,108.0,87.0,175.0


In [35]:
treatment_df = treatment_df.drop('c', axis=1)
treatment_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13600 entries, 0 to 13599
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   c                   13600 non-null  float64
 1   bill_id             13600 non-null  int64  
 2   patient_id          13600 non-null  object 
 3   date_of_admission   13600 non-null  object 
 4   amount              13600 non-null  float64
 5   gender              13600 non-null  object 
 6   race                13600 non-null  object 
 7   resident_status     13600 non-null  object 
 8   date_of_birth       13600 non-null  object 
 9   date_of_discharge   13600 non-null  object 
 10  medical_history_1   13600 non-null  int64  
 11  medical_history_2   13600 non-null  float64
 12  medical_history_3   13600 non-null  object 
 13  medical_history_4   13600 non-null  int64  
 14  medical_history_5   13600 non-null  float64
 15  medical_history_6   13600 non-null  int64  
 16  medi

In [ ]:
profile = ProfileReport(treatment_df, title='Pandas Profile Report')
profile

In [ ]:
treatment_df.info()